In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from scipy.stats import zscore
from sklearn.preprocessing import power_transform
# from statsmodels.tsa.stattools import adfuller
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
import pickle

# Data Loading

Source:
    https://www.kaggle.com/datasets/nicholasjhana/energy-consumption-generation-prices-and-weather

In [2]:
weather_pred=pd.read_csv('weather_features.csv')
weather_pred

,dt_iso,city_name,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,2015-01-01 00:00:00+01:00,Valencia,270.475,270.475,270.475,1001,77,1,62,0.0,0.0,0.0,0,800,clear,sky is clear,01n
1,2015-01-01 01:00:00+01:00,Valencia,270.475,270.475,270.475,1001,77,1,62,0.0,0.0,0.0,0,800,clear,sky is clear,01n
2,2015-01-01 02:00:00+01:00,Valencia,269.686,269.686,269.686,1002,78,0,23,0.0,0.0,0.0,0,800,clear,sky is clear,01n
3,2015-01-01 03:00:00+01:00,Valencia,269.686,269.686,269.686,1002,78,0,23,0.0,0.0,0.0,0,800,clear,sky is clear,01n
4,2015-01-01 04:00:00+01:00,Valencia,269.686,269.686,269.686,1002,78,0,23,0.0,0.0,0.0,0,800,clear,sky is clear,01n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178391,2018-12-31 19:00:00+01:00,Seville,287.760,287.150,288.150,1028,54,3,30,0.0,0.0,0.0,0,800,clear,sky is clear,01n
178392,2018-12-31 20:00:00+01:00,Seville,285.760,285.150,286.150,1029,62,3,30,0.0,0.0,0.0,0,800,clear,sky is clear,01n
178393,2018-12-31 21:00:00+01:00,Seville,285.150,285.150,285.150,1028,58,4,50,0.0,0.0,0.0,0,800,clear,sky is clear,01n
178394,2018-12-31 22:00:00+01:00,Seville,284.150,284.150,284.150,1029,57,4,60,0.0,0.0,0.0,0,800,clear,sky is clear,01n


In [3]:

#take a random sample of 20,000 rows
weather_pred=weather_pred.sample(n=20000, random_state=1)


# DataPreprocessing 

    1. Extracting date/time/month/year/hour/day/weekday/quarter

In [4]:
weather_pred['date']=pd.to_datetime(weather_pred['dt_iso'].str[:10])
weather_pred['time']=pd.to_datetime(weather_pred['dt_iso'].str[11:19])
weather_pred['month']=weather_pred['date'].dt.month
weather_pred['year']=weather_pred['date'].dt.year
weather_pred['hour']=weather_pred['time'].dt.hour
weather_pred['day']=weather_pred['date'].dt.day
weather_pred['weekday']=weather_pred['date'].dt.weekday
weather_pred['quarter']=weather_pred['date'].dt.quarter

In [5]:
weather_pred

,dt_iso,city_name,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,...,weather_description,weather_icon,date,time,month,year,hour,day,weekday,quarter
26385,2018-01-01 19:00:00+01:00,Valencia,288.1500,288.1500,288.1500,1027,47,7,270,0.0,...,sky is clear,01n,2018-01-01,2024-03-20 19:00:00,1,2018,19,1,0,1
127962,2017-04-30 04:00:00+02:00,Barcelona,286.1500,285.1500,287.1500,1013,87,0,170,0.0,...,overcast clouds,04n,2017-04-30,2024-03-20 04:00:00,4,2017,4,30,6,2
110320,2015-05-04 05:00:00+02:00,Barcelona,289.5425,289.5425,289.5425,1014,96,3,57,0.0,...,broken clouds,04,2015-05-04,2024-03-20 05:00:00,5,2015,5,4,0,2
158994,2016-10-29 21:00:00+02:00,Seville,295.0120,295.0120,295.0120,1029,55,1,42,0.0,...,sky is clear,01n,2016-10-29,2024-03-20 21:00:00,10,2016,21,29,5,4
100892,2018-04-12 07:00:00+02:00,Bilbao,281.4900,280.1500,283.1500,1002,93,4,290,0.3,...,light rain,10n,2018-04-12,2024-03-20 07:00:00,4,2018,7,12,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25461,2017-11-24 07:00:00+01:00,Valencia,287.1500,287.1500,287.1500,1019,78,3,290,0.0,...,few clouds,02n,2017-11-24,2024-03-20 07:00:00,11,2017,7,24,4,4
133922,2018-01-02 03:00:00+01:00,Barcelona,284.6500,284.1500,285.1500,1023,62,6,290,0.0,...,sky is clear,01n,2018-01-02,2024-03-20 03:00:00,1,2018,3,2,1,1
67939,2018-08-20 02:00:00+02:00,Madrid,297.5800,297.1500,298.1500,1021,36,2,360,0.0,...,sky is clear,01n,2018-08-20,2024-03-20 02:00:00,8,2018,2,20,0,3
74470,2015-05-08 11:00:00+02:00,Bilbao,290.4030,290.4030,290.4030,1016,55,1,150,0.0,...,few clouds,02,2015-05-08,2024-03-20 11:00:00,5,2015,11,8,4,2


In [6]:
weather_pred.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20000 entries, 26385 to 62361
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   dt_iso               20000 non-null  object        
 1   city_name            20000 non-null  object        
 2   temp                 20000 non-null  float64       
 3   temp_min             20000 non-null  float64       
 4   temp_max             20000 non-null  float64       
 5   pressure             20000 non-null  int64         
 6   humidity             20000 non-null  int64         
 7   wind_speed           20000 non-null  int64         
 8   wind_deg             20000 non-null  int64         
 9   rain_1h              20000 non-null  float64       
 10  rain_3h              20000 non-null  float64       
 11  snow_3h              20000 non-null  float64       
 12  clouds_all           20000 non-null  int64         
 13  weather_id           20000 non-n

In [7]:
# Prepare the data for analysis.
# Drop the columns that are not required for analysis.
weather_pred.drop(['dt_iso','city_name','weather_description','date','time'],axis=1,inplace=True)

In [8]:
weather_pred

,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id,weather_main,weather_icon,month,year,hour,day,weekday,quarter
26385,288.1500,288.1500,288.1500,1027,47,7,270,0.0,0.000,0.0,0,800,clear,01n,1,2018,19,1,0,1
127962,286.1500,285.1500,287.1500,1013,87,0,170,0.0,0.000,0.0,90,804,clouds,04n,4,2017,4,30,6,2
110320,289.5425,289.5425,289.5425,1014,96,3,57,0.0,0.000,0.0,58,803,clouds,04,5,2015,5,4,0,2
158994,295.0120,295.0120,295.0120,1029,55,1,42,0.0,0.000,0.0,0,800,clear,01n,10,2016,21,29,5,4
100892,281.4900,280.1500,283.1500,1002,93,4,290,0.3,0.000,0.0,75,500,rain,10n,4,2018,7,12,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25461,287.1500,287.1500,287.1500,1019,78,3,290,0.0,0.000,0.0,20,801,clouds,02n,11,2017,7,24,4,4
133922,284.6500,284.1500,285.1500,1023,62,6,290,0.0,0.000,0.0,0,800,clear,01n,1,2018,3,2,1,1
67939,297.5800,297.1500,298.1500,1021,36,2,360,0.0,0.000,0.0,0,800,clear,01n,8,2018,2,20,0,3
74470,290.4030,290.4030,290.4030,1016,55,1,150,0.0,0.085,0.0,12,801,clouds,02,5,2015,11,8,4,2


    2. Check for null values.

In [9]:
weather_pred.isnull().sum()

temp            0
temp_min        0
temp_max        0
pressure        0
humidity        0
wind_speed      0
wind_deg        0
rain_1h         0
rain_3h         0
snow_3h         0
clouds_all      0
weather_id      0
weather_main    0
weather_icon    0
month           0
year            0
hour            0
day             0
weekday         0
quarter         0
dtype: int64

    3. Check for duplicates

In [10]:
weather_pred.duplicated().sum()

0

    4. Encoding the columns

In [11]:
# label encoding
le=LabelEncoder()
weather_pred['weather_main']=le.fit_transform(weather_pred['weather_main'])
weather_pred['weather_icon']=le.fit_transform(weather_pred['weather_icon'])
weather_pred

,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id,weather_main,weather_icon,month,year,hour,day,weekday,quarter
26385,288.1500,288.1500,288.1500,1027,47,7,270,0.0,0.000,0.0,0,800,0,2,1,2018,19,1,0,1
127962,286.1500,285.1500,287.1500,1013,87,0,170,0.0,0.000,0.0,90,804,1,11,4,2017,4,30,6,2
110320,289.5425,289.5425,289.5425,1014,96,3,57,0.0,0.000,0.0,58,803,1,9,5,2015,5,4,0,2
158994,295.0120,295.0120,295.0120,1029,55,1,42,0.0,0.000,0.0,0,800,0,2,10,2016,21,29,5,4
100892,281.4900,280.1500,283.1500,1002,93,4,290,0.3,0.000,0.0,75,500,7,16,4,2018,7,12,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25461,287.1500,287.1500,287.1500,1019,78,3,290,0.0,0.000,0.0,20,801,1,5,11,2017,7,24,4,4
133922,284.6500,284.1500,285.1500,1023,62,6,290,0.0,0.000,0.0,0,800,0,2,1,2018,3,2,1,1
67939,297.5800,297.1500,298.1500,1021,36,2,360,0.0,0.000,0.0,0,800,0,2,8,2018,2,20,0,3
74470,290.4030,290.4030,290.4030,1016,55,1,150,0.0,0.085,0.0,12,801,1,3,5,2015,11,8,4,2


    5.  Removing Outliers

In [12]:
# Remove the outliers.
z=np.abs(zscore(weather_pred))
z

,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id,weather_main,weather_icon,month,year,hour,day,weekday,quarter
26385,0.186548,0.028462,0.343705,0.008271,0.973646,1.988401,0.885097,0.194370,0.076113,0.018761,0.815728,0.367488,0.598264,0.676867,1.597009,1.336529,1.095423,1.670489,1.485527,1.348524
127962,0.435173,0.404455,0.459683,0.010211,0.844167,1.078125,0.027970,0.194370,0.076113,0.018761,2.103934,0.404316,0.159348,0.897852,0.729313,0.440462,1.089916,1.612594,1.506873,0.455136
110320,0.013443,0.146062,0.182205,0.010072,1.253175,0.236101,0.940584,0.194370,0.076113,0.018761,1.065832,0.395109,0.159348,0.547915,0.440081,1.351673,0.944227,1.330860,1.485527,0.455136
158994,0.666484,0.831561,0.452137,0.007994,0.610083,0.640050,1.069153,0.194370,0.076113,0.018761,0.815728,0.367488,0.598264,0.676867,1.006079,0.455605,1.386802,1.499384,1.008140,1.331639
100892,1.014468,1.031111,0.923595,0.011735,1.116839,0.674176,1.056523,0.575563,0.076113,0.018761,1.617324,2.394684,2.474145,1.772696,0.729313,1.336529,0.652848,0.425182,0.010673,0.455136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25461,0.310860,0.153793,0.459683,0.009380,0.435159,0.236101,1.056523,0.194370,0.076113,0.018761,0.166914,0.376695,0.159348,0.151960,1.295311,0.440462,0.652848,0.933335,0.509406,1.331639
133922,0.621641,0.529786,0.691639,0.008826,0.291966,1.550326,1.056523,0.194370,0.076113,0.018761,0.815728,0.367488,0.598264,0.676867,1.597009,1.336529,1.235605,1.557280,0.986794,1.348524
67939,0.985719,1.099519,0.816077,0.009103,1.473544,0.201974,1.656512,0.194370,0.076113,0.018761,0.815728,0.367488,0.598264,0.676867,0.427615,1.336529,1.381295,0.480496,1.485527,0.438251
74470,0.093528,0.253909,0.082406,0.009795,0.610083,0.640050,0.143456,0.194370,18.372859,0.018761,0.426439,0.376695,0.159348,0.501898,0.440081,1.351673,0.070091,0.878021,0.509406,0.455136


In [13]:
threshold=3
print(np.where(z>3))

weather_pred_new=weather_pred[(z<3).all(axis=1)]
weather_pred_new

(array([    8,    51,    58,    87,   127,   149,   162,   173,   182,
         191,   224,   224,   335,   358,   362,   380,   380,   411,
         417,   445,   445,   445,   454,   469,   479,   507,   562,
         594,   609,   633,   638,   650,   677,   721,   746,   801,
         809,   813,   818,   825,   827,   830,   830,   830,   841,
         841,   875,   880,   882,   913,   932,   939,   954,   961,
         961,   975,  1057,  1065,  1069,  1074,  1114,  1169,  1173,
        1187,  1187,  1234,  1252,  1265,  1274,  1298,  1337,  1379,
        1400,  1400,  1407,  1415,  1494,  1498,  1588,  1593,  1593,
        1649,  1677,  1689,  1711,  1721,  1725,  1749,  1776,  1790,
        1803,  1803,  1808,  1817,  1837,  1853,  1939,  1973,  1979,
        2017,  2048,  2048,  2080,  2084,  2097,  2129,  2169,  2245,
        2256,  2256,  2257,  2277,  2303,  2316,  2347,  2354,  2371,
        2379,  2437,  2545,  2545,  2588,  2588,  2672,  2674,  2674,
        2699,  2721

,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id,weather_main,weather_icon,month,year,hour,day,weekday,quarter
26385,288.1500,288.1500,288.1500,1027,47,7,270,0.0,0.0,0.0,0,800,0,2,1,2018,19,1,0,1
127962,286.1500,285.1500,287.1500,1013,87,0,170,0.0,0.0,0.0,90,804,1,11,4,2017,4,30,6,2
110320,289.5425,289.5425,289.5425,1014,96,3,57,0.0,0.0,0.0,58,803,1,9,5,2015,5,4,0,2
158994,295.0120,295.0120,295.0120,1029,55,1,42,0.0,0.0,0.0,0,800,0,2,10,2016,21,29,5,4
100892,281.4900,280.1500,283.1500,1002,93,4,290,0.3,0.0,0.0,75,500,7,16,4,2018,7,12,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126651,290.1500,289.1500,291.1500,1020,41,4,260,0.0,0.0,0.0,0,800,0,1,3,2017,12,7,1,1
25461,287.1500,287.1500,287.1500,1019,78,3,290,0.0,0.0,0.0,20,801,1,5,11,2017,7,24,4,4
133922,284.6500,284.1500,285.1500,1023,62,6,290,0.0,0.0,0.0,0,800,0,2,1,2018,3,2,1,1
67939,297.5800,297.1500,298.1500,1021,36,2,360,0.0,0.0,0.0,0,800,0,2,8,2018,2,20,0,3


    6. Removing Skewness

In [14]:
weather_pred_new = pd.DataFrame(power_transform(weather_pred_new, method='yeo-johnson'), columns=weather_pred.columns)
weather_pred_new

,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,rain_3h,snow_3h,clouds_all,weather_id,weather_main,weather_icon,month,year,hour,day,weekday,quarter
0,-0.153266,0.003312,-0.306449,0.937768,-1.013988,1.855301,0.856624,-0.325423,-0.065902,-0.017683,-1.023142,0.325739,-0.978703,-0.560682,-1.720988,1.336498,1.063023,-1.889866,-1.608863,-1.402464
1,-0.409206,-0.379090,-0.428623,-0.390090,0.874785,-1.834663,0.247154,-0.325423,-0.065902,-0.017683,1.342769,0.633095,0.614478,1.139663,-0.698546,0.424104,-1.064820,1.494820,1.407157,-0.429306
2,0.021851,0.177095,-0.138790,-0.303986,1.368720,0.555521,-0.707094,-0.325423,-0.065902,-0.017683,1.136840,0.553568,0.614478,0.937163,-0.394557,-1.357162,-0.890047,-1.382454,-1.608863,-0.429306
3,0.686167,0.837841,0.493057,1.150500,-0.680478,-0.673171,-0.886543,-0.325423,-0.065902,-0.017683,-1.023142,0.325739,-0.978703,-0.560682,0.984078,-0.473707,1.301735,1.402707,0.995309,1.286352
4,-1.026554,-1.041748,-0.932636,-1.258058,1.201534,0.966004,0.965171,3.072265,-0.065902,-0.017683,1.257681,-2.672126,1.764315,1.510333,-0.698546,1.336498,-0.563426,-0.338264,0.099688,-0.429306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19189,0.097471,0.128346,0.051258,0.239983,-1.247461,0.966004,0.800971,-0.325423,-0.065902,-0.017683,-1.023142,0.325739,-0.978703,-1.134292,-1.017153,0.424104,0.165850,-0.958120,-0.955030,-1.402464
19190,-0.280576,-0.122928,-0.428623,0.145977,0.404405,0.555521,0.965171,-0.325423,-0.065902,-0.017683,0.628497,0.399929,0.614478,0.337007,1.239977,0.424104,-0.563426,0.928596,0.561491,1.286352
19191,-0.604662,-0.509045,-0.677517,0.530363,-0.369250,1.599187,0.965171,-0.325423,-0.065902,-0.017683,-1.023142,0.325739,-0.978703,-0.560682,-1.720988,1.336498,-1.249904,-1.705993,-0.955030,-1.402464
19192,0.985677,1.087016,0.837319,0.335370,-1.430293,0.037166,1.320449,-0.325423,-0.065902,-0.017683,-1.023142,0.325739,-0.978703,-0.560682,0.455129,1.336498,-1.448798,0.530192,-1.608863,0.457950


## Feature Selection

    Train/Test Splitting the Data

In [15]:
# columns: temp	temp_min, temp_max, pressure, humidity, wind_speed, wind_deg, rain_1h, rain_3h, snow_3h,clouds_all,	weather_id,weather_main,weather_icon,month,	year,hour,day,weekday,quarter

X= weather_pred_new.drop('temp',axis=1)
y= weather_pred_new['temp']

In [16]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

    Best feature Selection
        Using Random Forest Regressor

In [17]:
k=5
model = RandomForestRegressor()
model.fit(X, y)
importances = model.feature_importances_
selected_feature_indices = np.argsort(importances)[-k:]
selected_features = X.columns[selected_feature_indices]

In [18]:
selected_features

Index(['wind_deg', 'hour', 'humidity', 'temp_max', 'temp_min'], dtype='object')

# Model Training

In [19]:
X_train = X_train[selected_features]
X_test = X_test[selected_features]

In [20]:
# X_train

In [21]:
# X_test

    Apply Decision Tree

In [22]:
# apply decision tree
dt = DecisionTreeRegressor()
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
y_pred_dt



array([ 0.2209318 , -0.83123352,  1.16505291, ..., -2.67795128,
       -0.09009958,  0.00917092])

    Model Evaluation: Decision Tree

In [23]:
# CALCULATE ACCURACY
print('Accuracy score:',dt.score(X_test,y_test))
print('R2 score:',r2_score(y_test,y_pred_dt))
print('Mean Absolute Error:',mean_absolute_error(y_test,y_pred_dt))
print('Mean Squared Error:',mean_squared_error(y_test,y_pred_dt))
print('Root Mean Squared Error:',np.sqrt(mean_squared_error(y_test,y_pred_dt)))

Accuracy score: 0.9965594324301363
R2 score: 0.9965594324301363
Mean Absolute Error: 0.027894147427987917
Mean Squared Error: 0.0034264021637577507
Root Mean Squared Error: 0.05853547782121327


    Decision Tree (Optimised)

In [24]:
dt_op = DecisionTreeRegressor()

# Define the hyperparameter grid
parameters = {
    'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9],
    'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7, 8, 9],
    'min_samples_split': [1, 2, 3, 4, 5, 6, 7, 8, 9],
    'criterion': ['squared_error', 'friedman_mse', 'poisson']
}

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=dt_op, param_grid=parameters, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best parameters and best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Use the best model to make predictions
y_pred_dt_op = best_model.predict(X_test)



In [25]:
# applying  decision tree with best parameters
dt_op = DecisionTreeRegressor(criterion='squared_error', max_depth=9, min_samples_leaf=1, min_samples_split=2)
dt_op.fit(X_train, y_train)
y_pred_dt_op = dt_op.predict(X_test)
y_pred_dt_op

array([ 0.22083791, -0.77526026,  1.16235876, ..., -2.597009  ,
       -0.14896655,  0.01536798])

In [26]:

# CALCULATE ACCURACY
print('Accuracy score:',dt_op.score(X_test,y_test))
print('R2 score:',r2_score(y_test,y_pred_dt_op))
print('Mean Absolute Error:',mean_absolute_error(y_test,y_pred_dt_op))
print('Mean Squared Error:',mean_squared_error(y_test,y_pred_dt_op))
print('Root Mean Squared Error:',np.sqrt(mean_squared_error(y_test,y_pred_dt_op)))


Accuracy score: 0.9969516612570856
R2 score: 0.9969516612570856
Mean Absolute Error: 0.028518531866712934
Mean Squared Error: 0.0030357882100837514
Root Mean Squared Error: 0.05509798735057163


# Creating Pickle files

In [27]:
# Creating pickle files for Decision Tree Regressor (optimised)
filename_dt = 'weather_pred_dt.pkl'
pickle.dump(dt_op, open(filename_dt, 'wb'))


In [28]:
# load the model
loaded_model_dt = pickle.load(open(filename_dt, 'rb'))
result_dt = loaded_model_dt.score(X_test, y_test)
print(result_dt)

0.9969516612570856
